# Интро и полезная литература

Данный файл является скорее демонстрацией того, в чем суть Word2Vec и что он умеет делать. Этот файл - песочница, на которой проводились эксперименты.

Все важные функции отсюда будут в дальнейшем перенесены в папку helpers и использованы в дальнейшем анализе

### Литература

- [Word2vec в картинках](https://habr.com/ru/post/446530/). Статья на хабре от 2019 года.
- [Презентация](http://www.machinelearning.ru/wiki/images/b/b3/Word2Vec.pdf) (2017 год) с примерами реализации кода на библиотеке gensim
 
Так как Word2Vec - это уже не просто единичная реализация, а группа алгоритмов для получения векторных представлений слов, то существует масса различных пакетов на эту тему. Мы будем пользоваться gensim.

- [Gensim – Руководство для начинающих](https://webdevblog.ru/gensim-rukovodstvo-dlya-nachinajushhih/). Здесь нужно изучить ряд терминов, чтобы понимать принцип работы библиотеки. Статья 2019 года.
- [A Beginner’s Guide to Word Embedding with Gensim Word2Vec Model](https://towardsdatascience.com/a-beginners-guide-to-word-embedding-with-gensim-word2vec-model-5970fa56cc92). Статья 2019 года. По ней хорошо учиться писать код.

### Термины

- **Токен** обычно означает «слово»
- **Документ** обычно может относиться как к «предложению» так и к «абзацу»
- **Корпус** обычно представляет собой «собрание документов» в виде пакета слов или как его еще называют мешка слов 

# Код

In [122]:
TOKEN_TYPES = {
    'string': 'StringLiteral',
    'null': 'Null',
    'boolean': 'Boolean',
    'number': 'Numeric',
    'regExp': 'RegularExpression',
    'template': 'Template',
    
    'blockComments': 'BlockComment',
    'lineComments': 'LineComment',
    'identifier': 'Identifier',
    'reservedWord': 'Keyword',
    'punctuator': 'Punctuator',
};

In [92]:
import json
pathToTokenizeScriptsJson = '../data/parsed-scripts.json'
  
tokenizedSciptsJson = open(pathToTokenizeScriptsJson) 
tokenized_scripts = json.load(tokenizedSciptsJson) 

In [136]:
script = tokenized_scripts['file1082.js']

In [138]:
def filter_tokens_fo_Word2Vec(tokenized_script):
    filtered_tokens = []
    
    for token in tokenized_script:
        if token['type'] not in [
            TOKEN_TYPES['blockComments'], TOKEN_TYPES['lineComments'], TOKEN_TYPES['punctuator']
        ]:
            filtered_tokens.append(token['value'])
    
    return filtered_tokens

In [139]:
filter_tokens_fo_Word2Vec(script)

['function',
 'global',
 'factory',
 'if',
 'typeof',
 'module',
 '"object"',
 'typeof',
 'module',
 'exports',
 '"object"',
 'module',
 'exports',
 'global',
 'document',
 'factory',
 'global',
 'true',
 'function',
 'w',
 'if',
 'w',
 'document',
 'throw',
 'new',
 'Error',
 '"jQuery requires a window with a document"',
 'return',
 'factory',
 'w',
 'else',
 'factory',
 'global',
 'typeof',
 'window',
 '"undefined"',
 'window',
 'this',
 'function',
 'window',
 'noGlobal',
 'var',
 'arr',
 'var',
 'slice',
 'arr',
 'slice',
 'var',
 'concat',
 'arr',
 'concat',
 'var',
 'push',
 'arr',
 'push',
 'var',
 'indexOf',
 'arr',
 'indexOf',
 'var',
 'class2type',
 'var',
 'toString',
 'class2type',
 'toString',
 'var',
 'hasOwn',
 'class2type',
 'hasOwnProperty',
 'var',
 'trim',
 '""',
 'trim',
 'var',
 'support',
 'var',
 'document',
 'window',
 'document',
 'version',
 '"2.1.0 -ajax,-ajax/jsonp,-ajax/load,-ajax/parseJSON,-ajax/parseXML,-ajax/script,-ajax/var/nonce,-ajax/var/rquery,-ajax/

In [36]:
import pandas as pd
from gensim.models.word2vec import Word2Vec
from multiprocessing import cpu_count

In [64]:
sentences = pd.read_csv('News_Final.csv')['Headline'][:10000]
list_of_lists = [[str(word) for word in str(sentence).split()] for sentence in sentences]

Теперь бы нужно настроить гиперпараметры.

[Эта статья](https://habr.com/ru/post/446530/) говорит, что embedding_size устанавливают обычно на 300.

In [86]:
model = Word2Vec(
    # list of lists. Например: [["cat", "say", "meow"], ["dog", "say", "woof"]]
    sentences=list_of_lists,
    
    # The number of dimensions of the embeddings and the default is 100
    size=300, 
    
    #The maximum distance between a target word and words around the target word. The default window is 5.
    window=5,
    
    # финальный размер словаря
    max_final_vocab=10000, 
    
    # игнорировать слова с частотностью ниже, чем эта (регулируется автоматически, если установлено max_final_vocab)
    min_count = 0, 
    
    workers=cpu_count(),
    
    # The training algorithm, either CBOW(0) or skip gram(1). The default training algorithm is CBOW
    sg=0
)

In [87]:
model.wv.most_similar('Obama')

[('WASHINGTON', 0.9933357238769531),
 ('President', 0.992800772190094),
 ('Barack', 0.9923136830329895),
 ('LUMPUR"""President', 0.9922382235527039),
 ('PARIS', 0.9921869039535522),
 ('adviser', 0.9921643733978271),
 ('LUMPUR', 0.991817831993103),
 ('PALESTINE,', 0.9916163682937622),
 ('congratulated', 0.9915984869003296),
 ('toughened', 0.9909123182296753)]